In [ ]:
%pip install plotly

import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

df = pd.read_csv('train.csv')
df.head()

In [ ]:
#Fazendo uma cópia do Dataframe Lido
df = df.copy()

# Excluir as linhas com a idade dos entregadores vazia
# ( Conceitos de seleção condicional )
linhas_vazias = df['Delivery_person_Age'] != 'NaN '
df = df.loc[linhas_vazias, :].copy()

linhas_vazias = df['Road_traffic_density'] != 'NaN '
df = df.loc[linhas_vazias, :].copy()

linhas_vazias = df['City'] != 'NaN '
df = df.loc[linhas_vazias, :].copy()

linhas_vazias = df['Festival'] != 'NaN '
df = df.loc[linhas_vazias, :].copy()


# Conversao de texto/categoria/string para numeros inteiros
df['Delivery_person_Age'] = df['Delivery_person_Age'].astype( int )

# Conversao de texto/categoria/strings para numeros decimais
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype( float )

# Conversao de texto para data
df['Order_Date'] = pd.to_datetime( df['Order_Date'], format='%d-%m-%Y' )

#
linhas_vazias = df['multiple_deliveries'] != 'NaN '
df = df.loc[linhas_vazias, :]
df['multiple_deliveries'] = df['multiple_deliveries'].astype( int )

# removendo os espaços dentro de strings/texto/object
df.loc[:, 'ID'] = df.loc[:, 'ID'].str.strip()
df.loc[:, 'Road_traffic_density'] = df.loc[:, 'Road_traffic_density'].str.strip()
df.loc[:, 'Type_of_order'] = df.loc[:, 'Type_of_order'].str.strip()
df.loc[:, 'Type_of_vehicle'] = df.loc[:, 'Type_of_vehicle'].str.strip()
df.loc[:, 'City'] = df.loc[:, 'City'].str.strip()
df.loc[:, 'Festival'] = df.loc[:, 'Festival'].str.strip()

# Limpando a coluna de time taken
df.loc[:, 'Time_taken(min)'] = df.loc[:, 'Time_taken(min)'].str.split(' ').str[1]
# df['Time_taken(min)'] = df['Time_taken(min)'].apply( lambda x: x.split( '(min) ')[1] )
df['Time_taken(min)'] = df['Time_taken(min)'].astype( int )

df1 = df.copy()
df1.head()

## 1. Desenhar um gráfico de pizza com a média de idade dos entregadores por idade

In [34]:
df_aux = df1['Delivery_person_Age'].value_counts().reset_index()
df_aux.columns = ['Delivery_person_Age', 'Count']

# Create the pie chart
fig = px.pie(df_aux, values='Count', names='Delivery_person_Age', title='Distribution of Delivery Person Ages')
fig.show()

## 2. Desenhe um gráfico de linha, mostrando o número total de entregas diárias feitas por densidade de tráfego igual a “Lowˮ e “Jamˮ

In [35]:
cols = ['ID', 'Order_Date', 'Road_traffic_density']

df_filtered_traffic = df1.loc[df1['Road_traffic_density'].isin(['Low', 'Jam']), cols]

# Group by Order_Date and Road_traffic_density to count deliveries per day per traffic type
df_aux = df_filtered_traffic.groupby( ['Order_Date', 'Road_traffic_density'] ).count().reset_index()
df_aux.columns = ['Order_Date', 'Road_traffic_density', 'Total_Deliveries']

# Create the line chart
fig = px.line( df_aux, x='Order_Date', y='Total_Deliveries', color='Road_traffic_density',
             title='Total Daily Deliveries by Low and Jam Traffic Density' )
fig.show()

## 3. Desenhe um gráfico de barras, mostrando as avaliações médias das entregas por semana.

In [36]:
df1['week_of_year'] = df1['Order_Date'].dt.strftime( "%U" )

df_aux = df1.loc[:, ['Delivery_person_Ratings', 'week_of_year']].groupby( 'week_of_year' ).mean().reset_index()

# gráfico
px.bar( df_aux, x='week_of_year', y='Delivery_person_Ratings' )

## 4. Desenhe um gráfico de pizza, mostrando a média de avaliações das entregas feitas por condições climáticas?

In [37]:
cols = ['Delivery_person_Ratings', 'Weatherconditions']

df2 = df1.loc[:, cols].groupby(['Weatherconditions']).mean().reset_index()

# Create the pie chart
fig = px.pie(df2, values='Delivery_person_Ratings', names='Weatherconditions', title='Delivery_person_Ratings')
fig.show()